# This is the code for Quantum Optimization

In [63]:
import qiskit
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp
import qiskit_aer as aer
from docplex.mp.model import Model
from qiskit_algorithms import QAOA
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms.optimizers import COBYLA
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit_optimization.converters import QuadraticProgramToQubo

## We first create the Docplex/Cplex optimization problem

In [64]:
opt_model = Model(name="MIP Model")

Nc= 20 # Nc is the number of seconds

Dist=300 # Distance to travel

tolerance = 2 # Tolerance in distance travelled

delta_v=1 # Rate of acceleration/deceleration set to 1

vmax=150 # Max speed of a TGV in France (in m/s)

alpha=0.05 # Regenerative braking efficiency

"""
We define two binary variables for two bits. 
When x=0 and y=0 then constant velocity
When x=1 and y=0 then acceleration
When x=0 and y=1 then breaking
"""

x={}
for i in range(0, Nc):
    x[i]= opt_model.binary_var(name=f"x_{i}")

y={}
for i in range(0, Nc):
    y[i]= opt_model.binary_var(name=f"y_{i}")

z={}
for i in range(0, Nc):
    z[i]= opt_model.binary_var(name=f"z_{i}")
    

objective = opt_model.linear_expr()
## objective is the hamiltonian/energy value we want to minimize
## Energy:
for i in range(0, Nc):
    objective += (delta_v**2) * x[i] - alpha*(delta_v**2)*y[i]
    #objective += (delta_v**2)*x[i]

    
opt_model.minimize(objective)

    
## Constraint 1: (simultaneous braking/acceleration)


for i in range(0, Nc):
    opt_model.add_constraint(z[i] <= x[i] , f"z_u_d_{i}") 

for i in range(0, Nc):
    opt_model.add_constraint(z[i] <= y[i], f"z_p_d_{i}")
        
for i in range(0, Nc):
    opt_model.add_constraint(z[i] >= x[i]+ y[i] -1 , f"z_u_p_d_{i}") 
    
opt_model.add_constraint(opt_model.sum(z[i] for i in range(0, Nc)) == 0 , "No_simultaneous_braking_or_acceleration_constraint")

"""
Constraint 2: (Total Distance constraints)
"""

distance = opt_model.linear_expr()
velocity = 0
for i in range(0, Nc):
    velocity = velocity + delta_v*(x[i]-y[i])
    distance += velocity
opt_model.add_constraint(distance <= Dist+tolerance, "Max_Distance_constraint")
opt_model.add_constraint(distance >= Dist-tolerance, "Min_Distance_constraint")


"""
Constraint 3: (Net-Zero contraint)

"""
opt_model.add_constraint(opt_model.sum((y[i]-x[i]) for i in range(0, Nc)) == 0 , "Net_Zero_constraint")


"""
Constraint 4: (Maximum Speed)

"""
opt_model.add_constraint(opt_model.sum((delta_v*x[i]) for i in range(0, Nc)) <= vmax , "Maximum_Speed_constraint")

'''
#Constraint 5: (Positive Speed)
for i in range(Nc):
    opt_model.add_constraint(opt_model.sum((x[i]-y[i]) for i in range(0, i)) >= 0 , "Positive_Speed_constraint")


#Constraint 6: (Must leave immediately)

opt_model.add_constraint(x[0]==1, "Must leave immediately constraint")'''
    

#### Print the optimization model

opt_model.print_information() 

print(opt_model.prettyprint())

Model: MIP Model
 - number of variables: 60
   - binary=60, integer=0, continuous=0
 - number of constraints: 65
   - linear=65
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP
// This file has been generated by DOcplex
// model name is: MIP Model
// single vars section
dvar bool x_0;
dvar bool x_1;
dvar bool x_2;
dvar bool x_3;
dvar bool x_4;
dvar bool x_5;
dvar bool x_6;
dvar bool x_7;
dvar bool x_8;
dvar bool x_9;
dvar bool x_10;
dvar bool x_11;
dvar bool x_12;
dvar bool x_13;
dvar bool x_14;
dvar bool x_15;
dvar bool x_16;
dvar bool x_17;
dvar bool x_18;
dvar bool x_19;
dvar bool y_0;
dvar bool y_1;
dvar bool y_2;
dvar bool y_3;
dvar bool y_4;
dvar bool y_5;
dvar bool y_6;
dvar bool y_7;
dvar bool y_8;
dvar bool y_9;
dvar bool y_10;
dvar bool y_11;
dvar bool y_12;
dvar bool y_13;
dvar bool y_14;
dvar bool y_15;
dvar bool y_16;
dvar bool y_17;
dvar bool y_18;
dvar bool y_19;
dvar bool z_0;
dvar bool z_1;
dvar bool z_2;
dvar bool z_3;
dvar bool z_4;
dvar bool z

## Problem conversion

In [65]:
qp = from_docplex_mp(opt_model)
print(qp.prettyprint())

Problem name: MIP Model

Minimize
  x_0 + x_1 + x_10 + x_11 + x_12 + x_13 + x_14 + x_15 + x_16 + x_17 + x_18
  + x_19 + x_2 + x_3 + x_4 + x_5 + x_6 + x_7 + x_8 + x_9 - 0.05*y_0 - 0.05*y_1
  - 0.05*y_10 - 0.05*y_11 - 0.05*y_12 - 0.05*y_13 - 0.05*y_14 - 0.05*y_15
  - 0.05*y_16 - 0.05*y_17 - 0.05*y_18 - 0.05*y_19 - 0.05*y_2 - 0.05*y_3
  - 0.05*y_4 - 0.05*y_5 - 0.05*y_6 - 0.05*y_7 - 0.05*y_8 - 0.05*y_9

Subject to
  Linear constraints (65)
    -x_0 + z_0 <= 0  'z_u_d_0'
    -x_1 + z_1 <= 0  'z_u_d_1'
    -x_2 + z_2 <= 0  'z_u_d_2'
    -x_3 + z_3 <= 0  'z_u_d_3'
    -x_4 + z_4 <= 0  'z_u_d_4'
    -x_5 + z_5 <= 0  'z_u_d_5'
    -x_6 + z_6 <= 0  'z_u_d_6'
    -x_7 + z_7 <= 0  'z_u_d_7'
    -x_8 + z_8 <= 0  'z_u_d_8'
    -x_9 + z_9 <= 0  'z_u_d_9'
    -x_10 + z_10 <= 0  'z_u_d_10'
    -x_11 + z_11 <= 0  'z_u_d_11'
    -x_12 + z_12 <= 0  'z_u_d_12'
    -x_13 + z_13 <= 0  'z_u_d_13'
    -x_14 + z_14 <= 0  'z_u_d_14'
    -x_15 + z_15 <= 0  'z_u_d_15'
    -x_16 + z_16 <= 0  'z_u_d_16'
    -x_17 + 

In [66]:
qp2qubo = QuadraticProgramToQubo()
qubo = qp2qubo.convert(qp)
qubitOp, offset = qubo.to_ising()
print("Offset:", offset)
print("Ising Hamiltonian:")
print(str(qubitOp))

Offset: 216527485.50000048
Ising Hamiltonian:
SparsePauliOp(['...'],
              coeffs=[ 12375104.   +0.j, -12261045.475+0.j,  11762096.   +0.j, ...,
   1824768.   +0.j,    655776.   +0.j,   1311552.   +0.j])


In [67]:
from qiskit_ibm_runtime import QiskitRuntimeService
# Load saved credentials
service = QiskitRuntimeService()
# Get the least busy backend
backend = service.least_busy(operational=True, simulator=False)
print("least busy backend: ", backend)

least busy backend:  <IBMBackend('ibm_kyiv')>


In [68]:
sampler=Sampler(mode=backend)
optimizer=COBYLA(maxiter=100)
qaoa=QAOA(sampler=sampler, optimizer=optimizer)
meo = MinimumEigenOptimizer(qaoa)
result = meo.solve(qubitOp)
print(result.prettyprint())

AttributeError: 'SparsePauliOp' object has no attribute 'get_num_continuous_vars'